In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import EventsData
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
from shapely.geometry import LineString
from linref.events.geometry import LineStringM, linemerge_m
import shapely

l1 = shapely.from_wkt('LINESTRING (0 0, 1 0, 1 1, 2 1, 3 2)')
l2 = shapely.from_wkt('LINESTRING (3 2, 4 3, 5 3, 6 4, 7 5)')
l3 = shapely.from_wkt('LINESTRING (8 5, 8 6, 9 7, 10 8, 11 9)')
lm1 = LineStringM(l1, [1, 5, 20, 30, 40])
lm2 = LineStringM(l2, [40, 60, 70, 80, 90])
lm3 = LineStringM(l3, [100, 110, 120, 130, 140])

In [3]:
l12 = shapely.ops.linemerge([l1, l2])
l123 = shapely.ops.linemerge([l1, l2, l3])
l12.wkt, l123.wkt

('LINESTRING (0 0, 1 0, 1 1, 2 1, 3 2, 4 3, 5 3, 6 4, 7 5)',
 'MULTILINESTRING ((0 0, 1 0, 1 1, 2 1, 3 2, 4 3, 5 3, 6 4, 7 5), (8 5, 8 6, 9 7, 10 8, 11 9))')

In [7]:
linemerge_m([lm2, lm1], allow_multiple=True, squeeze=False)

[LINESTRING M (0.0 0.0 1, 1.0 0.0 5, 1.0 1.0 20, 2.0 1.0 30, 3.0 2.0 40, 4.0 3.0 60, 5.0 3.0 70, 6.0 4.0 80, 7.0 5.0 90) # linref compatibility approximation]

In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import EventsData
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
df_linear_1 = pd.DataFrame(index=[7, 6, 9, 4, 3, 2, 0, 1], data={
    'county': ['A', 'A', 'A', 'A', 'A' ,'B', 'B', 'B'],
    'route': ['Main St', 'Main St', 'Main St', '1st Ave', '1st Ave', '2nd Ave', '2nd Ave', 'Sycamore School Rd'],
    'beg': [0.0, 0.5, 1.1, 10.0, 12.0, 13.0, 13.1, 1.8],
    'end': [0.4, 1.1, 3.0, 12.0, 13.0, 13.1, 14.0, 3.0],
    'volume': [100, 200, 100, 400, 500, 150, 200, 800],
    'class': ['B', 'A', 'A', 'A', 'B', 'A', 'A', 'A'],
})
#df_linear_1 = pd.concat([df_linear_1] * 100, ignore_index=True)#.sort_values(['route', 'beg'])
df_linear_2 = pd.DataFrame({
    'route': ['Main St', 'Main St', '1st Ave', '2nd Ave', '2nd Ave', 'Sycamore School Rd'],
    'beg': [0.0, 1.0, 10.0, 13.0, 13.1, 2.2],
    'end': [1.0, 2.0, 12.0, 13.0, 14.0, 2.8],
    'street_type': ['A', 'B', 'A', 'A', 'B', 'A'],
})
#df_linear_2 = pd.concat([df_linear_2] * 100, ignore_index=True)#.sort_values(['route', 'beg'])

ed1 = EventsData(index=df_linear_1.index, groups=df_linear_1.route, begs=df_linear_1.beg, ends=df_linear_1.end, closed='left_mod')
ed2 = EventsData(index=df_linear_2.index, groups=df_linear_2.route, begs=df_linear_2.beg, ends=df_linear_2.end, closed='left_mod')
ed3 = EventsData(index=df_linear_1.index, groups=df_linear_1.route, locs=df_linear_1.beg)

In [3]:
import linref as lr
lr.set_default_lrs(key_col=['county', 'route'], loc_col='loc', beg_col='beg', end_col='end', geom_col='geometry', closed='left_mod')
df = pd.concat([df_linear_1] * 2000, ignore_index=True)

In [4]:
dfs = []
for i in range(2000):
    x = df_linear_1.copy()
    x['county'] = x['county'] + str(i)
    dfs.append(x)
df = pd.concat(dfs, ignore_index=True)

In [5]:
dissolved = df.lr.dissolve(retain=['class'])

In [6]:
er = dissolved.lr.relate(df)
arr = er.overlay()

In [17]:
er.sum(data=np.arange(32000).reshape(-1, 2), axis=1, squeeze=True)

array([[0.0000e+00, 1.0000e+00],
       [6.0000e+00, 8.0000e+00],
       [6.0000e+00, 7.0000e+00],
       ...,
       [3.1992e+04, 3.1993e+04],
       [6.3990e+04, 6.3992e+04],
       [3.1998e+04, 3.1999e+04]], shape=(12000, 2))